In [40]:
from matminer.featurizers.conversions import StrToComposition
from matminer.featurizers.composition import ElementProperty
from matminer.featurizers.base import MultipleFeaturizer
from matminer.featurizers import composition as cf
from matminer.featurizers.conversions import StrToComposition
from pymatgen.core.composition import Composition

import re
import numpy as np 
import pandas as pd 
from pandas.core.frame import DataFrame
from scipy import stats
from sklearn.ensemble import RandomForestClassifier
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import GridSearchCV
from sklearn.gaussian_process.kernels import WhiteKernel, ExpSineSquared
from sklearn.metrics import f1_score, matthews_corrcoef ,make_scorer,recall_score,roc_auc_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

import bz2 
import pickle 
import _pickle as cPickle
import os
import joblib
import argparse

In [41]:
feature_calculators = MultipleFeaturizer([
    cf.ElementProperty.from_preset(preset_name="magpie"),
    cf.Stoichiometry(),
    cf.ValenceOrbital(props=['frac']),
    cf.IonProperty(fast=True),
    cf.BandCenter(),
    cf.ElementFraction(),
    ])

def generate(fake_df, ignore_errors=False):
    """
        generate feature from a dataframe with a "formula" column that contains 
        chemical formulas of the compositions.
    """
    fake_df = StrToComposition().featurize_dataframe(fake_df, "formula", ignore_errors=ignore_errors)
    fake_df = fake_df.dropna()
    fake_df = feature_calculators.featurize_dataframe(fake_df, col_id='composition', ignore_errors=ignore_errors);
    fake_df["NComp"] = fake_df["composition"].apply(len)
    return fake_df

In [42]:
input = './data/predict.csv'
y = pd.read_csv(input).iloc[:,0].values
# print(input)
# # input='{1}{0}{1}'.format(input,"'")
# print(input)
formula = pd.read_csv(input)
# print(formula)
ext_magpie = generate(formula)
return(ext_magpie)
# ext_magpie('train.csv')

StrToComposition:   0%|          | 0/17367 [00:00<?, ?it/s]

MultipleFeaturizer:   0%|          | 0/17367 [00:00<?, ?it/s]

KeyError: "'U'\nTO SKIP THESE ERRORS when featurizing specific compounds, set 'ignore_errors=True' when running the batch featurize() operation (e.g., featurize_many(), featurize_dataframe(), etc.)."

In [27]:
input = './data/predict.csv'
y = pd.read_csv(input).iloc[:,0].values
# print(y)
ls = []
for i in y:
    comp=Composition(i)
    redu = comp.get_reduced_formula_and_factor()[1] # specific value ⇦ 具体的に何を指すか不明
    # redu_for = comp.get_reduced_formula_and_factor()[0]
    # redu_data=np.array(list(comp.as_dict().values()))
    most=comp.num_atoms # total atom number
    data=np.array(list(comp.as_dict().values()))
    # print(list(data))
    # l = len(data)
    # s = sum(data)
    # print(s)

    a = max(data) # maximum atom number
    #print(a)
    i = min(data) # minimum atom number
    m = np.mean(data) # average atom number
    # v = np.var(data)
    var = np.var(data/most) # atom number variance
    # var2 = np.var(data/redu)
    ls.append([most,a,i,m,redu,var,])

['Na8Al6Si6S1O28' 'Na8Al6Si6Br1Cl1O24' 'Na8Al6Si6I1Cl1O24' ...
 'Al16Si24W24O96' 'Sr24Fe16H96O96' 'Gd24Fe40O96']
